# Complementary slackness

The condition is called "complementary slackness". A rather generic definition is given in section 5.2.2 of the Boyd textbook (it also arises in linear programming), but more importantly, its impact for SDPs is described in Example 5.13.

In short, this provides a set of equations that are satisfied by the optimal solution of an SDP and its dual: in the notation of Example 5.13, if the primal is

  min c*x
  s.t. F(x) = sum_i x_i F_i + G <= 0

and the dual

  max tr(G*Z)
  s.t. tr(F_i*Z) + c_i = 0
    Z >= 0

then the optimal matrix solutions Fstar and Zstar span orthogonal vector spaces, in the sense that the product of both matrices

  (Fstar) * (Zstar) = 0
  1
is the null matrix.

I join here a short python example that illustrates this phenomenon with cvxpy. Note that the SDP data is chosen randomly in the script, so you can obtain other examples by removing the randomness seed setting, you may just need to try a few attempts because random problems may be either infeasible or unbounded.

By the way, since you found a meaningful basis of indeterminates for your problem, it might be interesting to work with the SDP matrices in the basis formed by your nullifiers. If Gamma is your matrix in the monomial basis (1, A0, A1, B0, ...), you should be able to construct this matrix by simply performing a change of basis U*Gamma*U^\dag. What is the structure of the matrix you obtain in this basis?

In [1]:
import cvxpy as cp
import numpy as np

# Problem size
m = 4                                             # m x m matrices
n = 5                                             # n scalar variables

# A random seed that works
np.random.seed(4)

# Define the problem
x = cp.Variable(n)                                # the variables
c = np.random.randn(n,1)                          # coefficients of the objective function
Fs = np.random.randn(m,m,n)
Fs = (Fs + Fs.transpose((1,0,2)))/2               # matrices should be symmetric
G = np.random.randn(m,m)
G = (G + G.transpose())/2
F = G + sum(x[i] * Fs[:, :, i] for i in range(n)) # the SDP matrix

# Construct the SDP
objective = cp.Minimize(c.transpose() @ x)
constraints = [F << 0]
prob = cp.Problem(objective, constraints)

# We attempt to solve the SDP
result = prob.solve()
if prob.status != 'optimal':
    print(prob.status + ', try again')
else:
    # The optimal value for x is stored in `x.value`.
    #print(x.value)
    # The optimal Lagrange multiplier for a constraint is stored in
    # `constraint.dual_value`.
    #print(constraints[0].dual_value)
    
    # We check the complementary slackness condition:
    Z = constraints[0].dual_value                 # The dual SDP matrix
    print('These terms should be zero by complementary slackness:')
    print(F.value @ Z)


These terms should be zero by complementary slackness:
[[ 3.08020297e-05  3.83227322e-05  9.53845473e-05  8.13472439e-05]
 [-3.83056612e-05 -5.68507917e-05 -1.63427572e-04 -1.33061511e-04]
 [ 2.00545141e-05  1.77073194e-05  2.67941801e-05  2.78275075e-05]
 [-1.71328273e-05 -8.49235084e-06  9.45183203e-06 -7.49088160e-07]]
